In [ ]:
!pip install --quiet datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 25.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.0 MB/s eta 0:00:00


In [ ]:
import torch
import pandas as pd
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer
import os
from accelerate import Accelerator
from tqdm import tqdm
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
import datasets

In [ ]:
df = pd.read_csv('/content/eng.csv')

In [ ]:
df['label'] = 0
df

,id,text,Anger,Fear,Joy,Sadness,Surprise,label
0,eng_train_track_a_00001,But not very happy.,0,0,1,1,0,0
1,eng_train_track_a_00002,Well she's not gon na last the whole song like...,0,0,1,0,0,0
2,eng_train_track_a_00003,She sat at her Papa's recliner sofa only to mo...,0,0,0,0,0,0
3,eng_train_track_a_00004,"Yes, the Oklahoma city bombing.",1,1,0,1,1,0
4,eng_train_track_a_00005,They were dancing to Bolero.,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...
2763,eng_train_track_a_02764,"""Yeah, but did you just find that?",0,1,0,0,1,0
2764,eng_train_track_a_02765,I did as little as possible with my right hand...,0,0,0,0,0,0
2765,eng_train_track_a_02766,"Okay that sucks, right?",1,0,0,1,0,0
2766,eng_train_track_a_02767,"The spark leaped through his body into mine, a...",0,1,0,0,1,0


In [ ]:
type(df.iloc[0]['Anger'])

numpy.int64

In [ ]:
df['labels'] = df.apply(lambda row: [
    float(row['Anger']), float(0), float(0),
    float(row['Fear']),
    float(row['Joy']), float(0), float(0), float(0),
    float(row['Sadness']),
    float(row['Surprise']), float(0)
], axis=1)
df = df.drop(columns=['id','Joy','Fear','Anger','Sadness','Surprise'])


In [ ]:
df = df.drop(columns=['label'])
df

,text,labels
0,But not very happy.,"[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, ..."
1,Well she's not gon na last the whole song like...,"[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ..."
2,She sat at her Papa's recliner sofa only to mo...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,"Yes, the Oklahoma city bombing.","[1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, ..."
4,They were dancing to Bolero.,"[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ..."
...,...,...
2763,"""Yeah, but did you just find that?","[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2764,I did as little as possible with my right hand...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2765,"Okay that sucks, right?","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ..."
2766,"The spark leaped through his body into mine, a...","[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [ ]:
from datasets import Dataset

dataset = Dataset.from_pandas(df)  # Use only the desired columns

# Optionally, print the dataset to check
print(dataset)

Dataset({
    features: ['text', 'labels'],
    num_rows: 2768
})


In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("cardiffnlp/twitter-roberta-large-emotion-latest", device_map="auto")

def preprocess_function(examples):
    # Tokenize the text
    inputs = tokenizer(examples["text"], truncation=True, padding="max_length", max_length=256)

    # Convert labels to float to match the expected input type for BCEWithLogitsLoss
    inputs["labels"] = [torch.tensor(label, dtype=torch.float) for label in examples["labels"]]

    return inputs

encoded_dataset = dataset.map(preprocess_function, batched=True)
encoded_dataset

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.33k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

Map:   0%|          | 0/2768 [00:00<?, ? examples/s]

Dataset({
    features: ['text', 'labels', 'input_ids', 'attention_mask'],
    num_rows: 2768
})

In [ ]:
print(encoded_dataset[0]["labels"])

[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0]


In [ ]:
encoded_dataset.set_format("torch", columns=["input_ids", "attention_mask", "labels"])

In [ ]:
print(encoded_dataset[0]["labels"])  # Should output a tensor of floats: e.g., tensor([1., 0., 0., ..., 0., 1.])


tensor([0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0.])


In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("cardiffnlp/twitter-roberta-large-emotion-latest", device_map="auto")
model

config.json:   0%|          | 0.00/1.22k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 1024, padding_idx=1)
      (position_embeddings): Embedding(514, 1024, padding_idx=1)
      (token_type_embeddings): Embedding(1, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-23): 24 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
 

In [ ]:
dataset = encoded_dataset.train_test_split(test_size=0.2)

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'labels', 'input_ids', 'attention_mask'],
        num_rows: 2214
    })
    test: Dataset({
        features: ['text', 'labels', 'input_ids', 'attention_mask'],
        num_rows: 554
    })
})

In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    # Sigmoid is typically used for multi-label classification to get probabilities
    probs = torch.sigmoid(torch.tensor(logits)).numpy()
    # Convert probabilities to binary predictions (0 or 1)
    predictions = (probs > 0.5).astype(int)

    # Calculate metrics
    f1 = f1_score(labels, predictions, average='macro', zero_division=1)
    precision = precision_score(labels, predictions, average='macro', zero_division=1)
    recall = recall_score(labels, predictions, average='macro', zero_division=1)
    accuracy = accuracy_score(labels, predictions)  # Multi-label accuracy

    return {
        "accuracy": accuracy,
        "f1": f1,
        "precision": precision,
        "recall": recall
    }

In [ ]:
from transformers import TrainingArguments, Trainer
# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=5,
    weight_decay=0.01,
    report_to="none"   # Disable wandb logging if not needed
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    compute_metrics=compute_metrics
)

In [ ]:
# Train the model
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,No log,0.238736,0.494585,0.880808,0.902156,0.865042
2,0.021000,0.248147,0.507220,0.889519,0.883202,0.897747
3,0.021000,0.266825,0.489170,0.886371,0.877679,0.895759
4,0.013400,0.265068,0.509025,0.886976,0.893581,0.882890
5,0.013400,0.267623,0.512635,0.891863,0.890752,0.894479


TrainOutput(global_step=1385, training_loss=0.013361213267495056, metrics={'train_runtime': 1944.686, 'train_samples_per_second': 5.692, 'train_steps_per_second': 0.712, 'total_flos': 5158396953492480.0, 'train_loss': 0.013361213267495056, 'epoch': 5.0})

In [ ]:
df = pd.read_csv('/content/eng.csv')

In [ ]:
true_labels = []
for row in range(len(df)):
    true_label = [int(df.loc[row]['Joy']),
                  int(df.loc[row]['Fear']),
                  int(df.loc[row]['Anger']),
                  int(df.loc[row]['Sadness']),
                  int(df.loc[row]['Surprise'])]
    true_labels.append(true_label)
print(true_labels)

[[1, 0, 0, 1, 0], [1, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 1, 1, 1, 1], [1, 0, 0, 0, 0], [1, 0, 0, 0, 0], [0, 1, 0, 1, 0], [0, 0, 1, 0, 0], [0, 0, 0, 0, 0], [0, 1, 0, 0, 1], [1, 0, 0, 0, 0], [0, 0, 0, 0, 0], [1, 0, 0, 0, 0], [1, 0, 0, 0, 0], [1, 0, 0, 0, 0], [0, 1, 0, 1, 0], [0, 1, 0, 0, 1], [1, 1, 0, 0, 1], [0, 1, 0, 1, 0], [1, 0, 0, 0, 0], [0, 1, 0, 0, 1], [1, 0, 1, 0, 1], [0, 1, 0, 1, 0], [0, 1, 0, 1, 1], [0, 1, 0, 0, 1], [1, 0, 0, 0, 0], [1, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 0, 1, 0], [0, 1, 0, 0, 0], [0, 1, 0, 1, 1], [0, 1, 0, 1, 0], [0, 1, 0, 1, 0], [0, 1, 0, 1, 0], [0, 1, 0, 0, 0], [0, 1, 1, 1, 0], [0, 1, 0, 1, 1], [0, 1, 0, 0, 0], [0, 1, 0, 1, 0], [1, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 1, 0, 0, 1], [0, 1, 0, 0, 1], [0, 1, 1, 0, 0], [0, 1, 0, 1, 0], [1, 0, 0, 0, 0], [1, 0, 0, 1, 0], [0, 0, 0, 0, 0], [1, 0, 0, 0, 0], [0, 1, 0, 0, 1], [0, 1, 0, 0, 0], [0, 1, 0, 0, 1], [0, 1, 0, 0, 0], [0, 0, 0, 0, 1], [0, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 1, 0, 0, 

In [ ]:
emotion_classifier = pipeline("text-classification", model=model, tokenizer=tokenizer, top_k=None)
correct = 0
pred_labels = []
for row in tqdm(range(len(df))):
    joy, fear, anger, sadness, surprise = 0,0,0,0,0


    # Define some sample text for emotion classification
    text = df.loc[row]["text"]
    # print(text)
    # Get emotion predictions
    predictions = emotion_classifier(text)
    # print(predictions)

    # Display the results
    for prediction in predictions[0]:
        # print(prediction['score'])
        # prediction['score'] = 1 if prediction['score']>0.5 else 0
        if prediction['label'] == 'joy' and prediction['score'] > 0.5:
            # print("Inside")
            joy = 1
        # else: joy = 0
        if prediction['label'] == 'fear' and prediction['score'] > 0.5:
            # print("Inside")
            fear = 1
        # else: fear = 0
        if prediction['label'] == 'anger' and prediction['score'] > 0.5:
            # print("Inside")
            anger = 1
        # else: anger = 0
        if prediction['label'] == 'sadness' and prediction['score'] > 0.5:
            # print("Inside")
            sadness = 1
        # else: sadness = 0
        if prediction['label'] == 'surprise' and prediction['score'] > 0.5:
            # print("Inside")
            surprise = 1
        # else: surprise = 0
    # print(joy, fear, anger, sadness, surprise )
    if (df.loc[row]['Joy'] == joy and
        df.loc[row]['Fear'] == fear and
        df.loc[row]['Anger'] == anger and
        df.loc[row]['Sadness'] == sadness and
        df.loc[row]['Surprise'] == surprise):
        correct = correct + 1
    pred_labels.append([joy, fear, anger, sadness, surprise])
print(correct/len(df))

        # print(f"Emotion: {prediction['label']}, Score: {prediction['score']:.4f}")

100%|██████████| 2768/2768 [01:20<00:00, 34.36it/s]

0.9024566473988439


In [ ]:
precision_micro = precision_score(true_labels, pred_labels, average='micro')
recall_micro = recall_score(true_labels, pred_labels, average='micro')

# average='macro': Calculate metrics for each label, then average the results equally
precision_macro = precision_score(true_labels, pred_labels, average='macro')
recall_macro = recall_score(true_labels, pred_labels, average='macro')

# average='weighted': Calculate metrics for each label, then average the results weighted by the number of true instances for each label
precision_weighted = precision_score(true_labels, pred_labels, average='weighted')
recall_weighted = recall_score(true_labels, pred_labels, average='weighted')

print(f"Micro Precision: {precision_micro}")
print(f"Micro Recall: {recall_micro}")

print(f"Macro Precision: {precision_macro}")
print(f"Macro Recall: {recall_macro}")

print(f"Weighted Precision: {precision_weighted}")
print(f"Weighted Recall: {recall_weighted}")

Micro Precision: 0.9528604118993135
Micro Recall: 0.960553633217993
Macro Precision: 0.9520606341479709
Macro Recall: 0.9544491611451577
Weighted Precision: 0.9529268728556715
Weighted Recall: 0.960553633217993


In [ ]:
accuracy = accuracy_score(true_labels, pred_labels)
micro_f1 = f1_score(true_labels, pred_labels, average='micro')
macro_f1 = f1_score(true_labels, pred_labels, average='macro')
weighted_f1 = f1_score(true_labels, pred_labels, average='weighted')
print(f"Accuracy: {accuracy*100}")
print(f"Micro F1 Score: {micro_f1}")
print(f"Macro F1 Score: {macro_f1}")
print(f"Weighted F1 Score: {weighted_f1}")

Accuracy: 90.24566473988439
Micro F1 Score: 0.95669155657668
Macro F1 Score: 0.9531757449600363
Weighted F1 Score: 0.9566661671347719


In [ ]:
df = pd.read_csv('/content/eng_a.csv')

In [ ]:
# Inferencing on dev set and storing the predicted outputs
emotion_classifier = pipeline("text-classification", model=model, tokenizer=tokenizer, top_k=None)
pred_labels = []
for row in tqdm(range(len(df))):
    joy, fear, anger, sadness, surprise = 0,0,0,0,0


    # Define some sample text for emotion classification
    text = df.loc[row]["text"]
    # print(text)
    # Get emotion predictions
    predictions = emotion_classifier(text)
    # print(predictions)

    # Display the results
    for prediction in predictions[0]:
        # print(prediction['score'])
        # prediction['score'] = 1 if prediction['score']>0.5 else 0
        if prediction['label'] == 'joy' and prediction['score'] > 0.5:
            # print("Inside")
            joy = 1
        # else: joy = 0
        if prediction['label'] == 'fear' and prediction['score'] > 0.5:
            # print("Inside")
            fear = 1
        # else: fear = 0
        if prediction['label'] == 'anger' and prediction['score'] > 0.5:
            # print("Inside")
            anger = 1
        # else: anger = 0
        if prediction['label'] == 'sadness' and prediction['score'] > 0.5:
            # print("Inside")
            sadness = 1
        # else: sadness = 0
        if prediction['label'] == 'surprise' and prediction['score'] > 0.5:
            # print("Inside")
            surprise = 1
        # else: surprise = 0
    # print(joy, fear, anger, sadness, surprise )

    pred_labels.append([joy, fear, anger, sadness, surprise])


        # print(f"Emotion: {prediction['label']}, Score: {prediction['score']:.4f}")

100%|██████████| 116/116 [00:03<00:00, 35.17it/s]


In [ ]:
pred_labels

[[0, 1, 0, 0, 1],
 [0, 1, 0, 0, 1],
 [0, 0, 0, 0, 1],
 [0, 0, 0, 1, 0],
 [0, 1, 0, 0, 1],
 [0, 1, 0, 0, 0],
 [0, 0, 1, 0, 0],
 [0, 0, 0, 1, 0],
 [1, 0, 0, 0, 0],
 [0, 1, 0, 1, 0],
 [1, 0, 0, 0, 0],
 [0, 1, 0, 1, 0],
 [0, 0, 1, 1, 1],
 [0, 1, 0, 0, 1],
 [1, 0, 0, 0, 0],
 [1, 1, 0, 0, 0],
 [0, 1, 1, 0, 0],
 [0, 1, 0, 0, 1],
 [0, 1, 0, 1, 1],
 [0, 1, 0, 0, 0],
 [1, 1, 0, 0, 1],
 [0, 1, 0, 1, 0],
 [0, 0, 1, 0, 0],
 [0, 1, 1, 0, 1],
 [0, 1, 0, 0, 0],
 [0, 0, 0, 0, 1],
 [0, 1, 0, 0, 1],
 [0, 0, 0, 0, 0],
 [1, 0, 0, 0, 0],
 [0, 1, 0, 0, 1],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 1, 1, 1, 0],
 [0, 1, 0, 1, 0],
 [0, 1, 0, 0, 0],
 [1, 0, 0, 0, 0],
 [0, 1, 0, 0, 0],
 [0, 1, 0, 1, 0],
 [1, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [1, 1, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 1, 1, 1, 0],
 [1, 0, 0, 0, 0],
 [0, 1, 1, 0, 0],
 [0, 1, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 1, 0, 0, 1],
 [0, 1, 0, 1, 0],
 [0, 0, 0, 0, 0],
 [0, 1, 0, 1, 0],
 [0, 1, 1, 0, 1],
 [0, 0, 0, 0, 0],
 [0, 1, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 1, 0,

In [ ]:
df

,id,text,Anger,Fear,Joy,Sadness,Surprise
0,eng_dev_track_a_00001,"My mouth fell open `` No, no, no... I..",NaN,NaN,NaN,NaN,NaN
1,eng_dev_track_a_00002,You can barely make out your daughter's pale f...,NaN,NaN,NaN,NaN,NaN
2,eng_dev_track_a_00003,But after blinking my eyes for a few times lep...,NaN,NaN,NaN,NaN,NaN
3,eng_dev_track_a_00004,Slowly rising to my feet I came to the conclus...,NaN,NaN,NaN,NaN,NaN
4,eng_dev_track_a_00005,I noticed this months after moving in and doin...,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
111,eng_dev_track_a_00112,"""ARcH stop your progression.",NaN,NaN,NaN,NaN,NaN
112,eng_dev_track_a_00113,"This 'star', starts to move across the sky.",NaN,NaN,NaN,NaN,NaN
113,eng_dev_track_a_00114,and my feet hurt.,NaN,NaN,NaN,NaN,NaN
114,eng_dev_track_a_00115,so i cried my eyes out and did the drawing.,NaN,NaN,NaN,NaN,NaN


In [ ]:
df_val = pd.DataFrame(pred_labels, columns=['Anger', 'Fear', 'Joy', 'Sadness', 'Surprise'])
df_val.to_csv('my_list.csv', index=False)

In [ ]:
df_val.insert(loc=0, column='id', value='text')

In [ ]:
df_val

,id,Anger,Fear,Joy,Sadness,Surprise
0,text,0,1,0,0,1
1,text,0,1,0,0,1
2,text,0,0,0,0,1
3,text,0,0,0,1,0
4,text,0,1,0,0,1
...,...,...,...,...,...,...
111,text,0,1,1,0,1
112,text,0,1,0,0,1
113,text,0,1,0,0,0
114,text,0,0,0,1,0


In [ ]:
df_val['id'] = [f'eng_dev_track_a_{i+1:05}' for i in range(len(df_val))]
df_val

,id,Anger,Fear,Joy,Sadness,Surprise
0,eng_dev_track_a_00001,0,1,0,0,1
1,eng_dev_track_a_00002,0,1,0,0,1
2,eng_dev_track_a_00003,0,0,0,0,1
3,eng_dev_track_a_00004,0,0,0,1,0
4,eng_dev_track_a_00005,0,1,0,0,1
...,...,...,...,...,...,...
111,eng_dev_track_a_00112,0,1,1,0,1
112,eng_dev_track_a_00113,0,1,0,0,1
113,eng_dev_track_a_00114,0,1,0,0,0
114,eng_dev_track_a_00115,0,0,0,1,0


In [ ]:
df_val.to_csv('my_pred_after_FT_twitterRoberta.csv', index=False)